In [ ]:
import numpy as np
import os
import signalflow as sf
from PIL import Image
from IPython.display import Audio, display
from pixasonics.core import App, Mapper
from pixasonics.features import MeanPixelValue
from pixasonics.synths import Theremin

# Create app
app = App()

In [ ]:
# load an image
img_path = "images/cellular_dataset/merged_8bit/Timepoint_005_220518-ST_C03_s1.jpg"
# img_path = "images/test.jpg"
img = app.load_image_file(img_path)

In [ ]:
# load image as a numpy array
img_path = "images/cellular_dataset/merged_8bit/Timepoint_005_220518-ST_C03_s1.jpg"
img = Image.open(img_path)
img = np.array(img)
app.load_image_data(img) # load as numpy array

In [ ]:
# load HDR images
img_path = "images/cellular_dataset/single_channel_16bit/Timepoint_005_220518-ST_C03_s1_w1.TIF"
app.load_image_file(img_path)

In [ ]:
# combine two HDR images and load as numpy array
img_path = "images/cellular_dataset/single_channel_16bit/Timepoint_005_220518-ST_C03_s1_w2.TIF"
img_path2 = "images/cellular_dataset/single_channel_16bit/Timepoint_005_220518-ST_C03_s1_w1.TIF"
img = Image.open(img_path)
img2 = Image.open(img_path2)
img = np.array(img)
img2 = np.array(img2)
img = np.stack([img, img2], axis=-1)
print(img.shape)
app.load_image_data(img) # load as numpy array

In [ ]:
# read all red-ch images into arrays and concatenate them in the channel dimension
img_folder = "images/cellular_dataset/single_channel_16bit/"
img_files = os.listdir(img_folder)
img_files = [f for f in img_files if f.endswith("w2.TIF")] # only red channel images
imgs = []
for img_file in img_files:
    img_path = os.path.join(img_folder, img_file)
    img = Image.open(img_path)
    img = np.array(img)
    imgs.append(img)
img = np.stack(imgs, axis=-1) # now the last dimension is the channel dimension
print(img.shape)
app.load_image_data(img) # load as numpy array

In [ ]:
# read all red-ch images into arrays and concatenate them in the layer dimension
img_folder = "images/cellular_dataset/single_channel_16bit/"
img_files = os.listdir(img_folder)
img_files = [f for f in img_files if f.endswith("w2.TIF")] # only red channel images
imgs = []
for img_file in img_files:
    img_path = os.path.join(img_folder, img_file)
    img = Image.open(img_path)
    img = np.array(img)[..., None] # add a new dimension for channels
    imgs.append(img)
img = np.stack(imgs, axis=-1) # now the last dimension is the layer dimension
print(img.shape)
app.load_image_data(img) # load as numpy array

In [ ]:
# combine red and green channels and all layers
img_folder = "images/cellular_dataset/single_channel_16bit/"
img_files = os.listdir(img_folder)
imgs_red = [f for f in img_files if f.endswith("w2.TIF")] # only red channel images
imgs_green = [f for f in img_files if f.endswith("w1.TIF")] # only green channel images
imgs = []
for img_red, img_green in zip(imgs_red, imgs_green):
    img_path_red = os.path.join(img_folder, img_red)
    img_path_green = os.path.join(img_folder, img_green)
    img_red = Image.open(img_path_red)
    img_green = Image.open(img_path_green)
    img_red = np.array(img_red)
    img_green = np.array(img_green)
    img = np.stack([img_red, img_green], axis=-1) # now the last dimension is the channel dimension
    imgs.append(img)
img = np.stack(imgs, axis=-1) # now the last dimension is the layer dimension
print(img.shape)
app.load_image_data(img) # load as numpy array

In [ ]:
# now use all images in the folder
img_folder = "images/cellular_dataset/single_channel_16bit/"
img_files = os.listdir(img_folder)
imgs_red = [f for f in img_files if f.endswith("w2.TIF")] # only red channel images
imgs_green = [f for f in img_files if f.endswith("w1.TIF")] # only green channel images
imgs_blue = [f for f in img_files if f.endswith("w3.TIF")] # only blue channel images
imgs = []
for img_red, img_green, img_blue in zip(imgs_red, imgs_green, imgs_blue):
    img_path_red = os.path.join(img_folder, img_red)
    img_path_green = os.path.join(img_folder, img_green)
    img_path_blue = os.path.join(img_folder, img_blue)
    img_red = Image.open(img_path_red)
    img_green = Image.open(img_path_green)
    img_blue = Image.open(img_path_blue)
    img_red = np.array(img_red)
    img_green = np.array(img_green)
    img_blue = np.array(img_blue)
    img = np.stack([img_red, img_green, img_blue], axis=-1) # now the last dimension is the channel dimension
    imgs.append(img)
img = np.stack(imgs, axis=-1) # now the last dimension is the layer dimension
print(img.shape)
app.load_image_data(img) # load as numpy array

In [5]:
app.normalize_display = False
app.normalize_display_global = False

In [3]:
# Create feature
mean_pix = MeanPixelValue()
# mean_pix = MeanPixelValue(selected_channels=[0, 1]) # test selected channels

In [ ]:
app.features

In [ ]:
app.attach_feature(mean_pix)
app.features

In [ ]:
# Create synth
# theremin = Theremin()
theremin = Theremin(frequency=[440, 440], panning=[-1, 1]) # test multichannel

In [ ]:
app.synths

In [ ]:
app.attach_synth(theremin)
app.synths

In [ ]:
# Create mapper
pix2freq = Mapper(mean_pix, theremin["frequency"], exponent=2, out_high=1000)

In [ ]:
app.mappers

In [ ]:
app.attach_mapper(pix2freq)
app.mappers

In [ ]:
app.detach_mapper(pix2freq)
app.mappers

In [ ]:
# test adding another mapping: mean pixel value to amplitude with an exponential curve
pix2amp = Mapper(mean_pix, theremin["amplitude"], exponent=2)
app.attach_mapper(pix2amp)

In [ ]:
pix2pan = Mapper(mean_pix, theremin["panning"])
app.attach_mapper(pix2pan)

In [ ]:
# detach a mapper
app.detach_mapper(pix2amp)
app.mappers

In [ ]:
# detach old frequency mapping and add a new one with different scaling
app.detach_mapper(pix2freq)
pix2freq = Mapper(mean_pix, theremin["frequency"], exponent=2, out_low=100, out_high=2000)
app.attach_mapper(pix2freq)
app.mappers

In [ ]:
graph = sf.AudioGraph.get_shared_graph()
print(graph.structure)
print(graph.status)

# Non-Real-Time Rendering

In [ ]:
# example: horizontal scan
duration = 5
my_timeline = [
    (0, {
        "probe_width": 1,
        "probe_height": 500,
        "probe_x": 0,
        "probe_y": 0
    }),
    (duration, {
        "probe_x": 499
    })
]

target_filename = "horizontal_scan.wav"

app.render_timeline_to_file(my_timeline, target_filename)

display(Audio(target_filename))

In [ ]:
# example: vertical scan
duration = 5
my_timeline = [
    (0, {
        "probe_width": 500,
        "probe_height": 1,
        "probe_x": 0,
        "probe_y": 0
    }),
    (duration, {
        "probe_y": 499
    })
]

target_filename = "vertical_scan.wav"

app.render_timeline_to_file(my_timeline, target_filename)

display(Audio(target_filename))

# Array reading proto

In [11]:
import numpy as np
from PIL import Image

In [ ]:
a = np.random.rand(100, 100, 20, 10)
a.shape

In [ ]:
selected_channels = [0, 2, 5, 9]
b = a[:, :, selected_channels, :]
b.shape

In [ ]:
np.min(a, axis=(1, 2)).shape

In [32]:
path_16bit = "images/cellular_dataset/single_channel_16bit/Timepoint_005_220518-ST_C03_s1_w1.TIF"
img = Image.open(path_16bit)

In [29]:
path_8bit = "images/cellular_dataset/merged_8bit/Timepoint_005_220518-ST_C03_s1.jpg"
img = Image.open(path_8bit)

In [ ]:
# get the image dimensions
width, height = img.size
# get image channels
channels = len(img.getbands())
width, height, channels

In [ ]:
# get the image data type
img.mode

In [ ]:
img = np.array(img)
img.shape, img.dtype

In [ ]:
# if the image is not dtype=uint8,divide it by the max of the dtype and convert to uint8
img = (img / np.iinfo(img.dtype).max * 255).astype(np.uint8)
img.shape, img.dtype


In [ ]:
np.iinfo(np.uint16).max, np.iinfo(np.uint8).max, np.finfo(np.float32).max

In [ ]:
# get the max of float32
np.finfo(np.float32).max

In [ ]:
# get whether the img is integer or float dtype
is_int = np.issubdtype(img.dtype, np.integer)
is_int

In [37]:
def load_image(path, target_width, target_height):
    img = Image.open(path)
    img = img.resize((target_width, target_height))
    img_hires = np.array(img)
    if np.issubdtype(img_hires.dtype, np.integer):
        img_display = (img_hires / np.iinfo(img_hires.dtype).max * 255).astype(np.uint8)
    else:
        img_display = (img_hires / np.finfo(img_hires.dtype).max * 255).astype(np.uint8)
    return img_hires, img_display

In [ ]:
bg_hires, bg_display = load_image(path_8bit, 500, 500)